# Bsky integration

In [ ]:
//| export

import { BskyXRPC } from "@mary/bluesky-client";
import type {
  AppBskyFeedDefs,
  AppBskyFeedPost,
} from "@mary/bluesky-client/lexicons";

type Thread = AppBskyFeedDefs.ThreadViewPost;
type Post = AppBskyFeedDefs.PostView;
type PostRecord = AppBskyFeedPost.Record;

const rpc = new BskyXRPC({ service: "https://public.api.bsky.app" });

# Post URL to @at protocol URI

Figure out how to convert bsky.app URL to a URI we can use to fetch data. We
need to go from this:

```
https://bsky.app/profile/callmephilip.com/post/3lcskn64bss2d
```

To smth like this

```
at://did:plc:ubdeopbbkbgedccgbum7dhsh/app.bsky.feed.post/3lcskn64bss2d"
```

`did:plc:ubdeopbbkbgedccgbum7dhsh` is DID for callmephilip.com handle, which
needs to be resolved first

In [ ]:
//| export

const postURLToAtpURI = async (
  postUrl: string,
): Promise<string> => {
  const urlParts = new URL(postUrl);
  const pathParts = urlParts.pathname.split("/");
  const h = await rpc.get("com.atproto.identity.resolveHandle", {
    params: {
      handle: pathParts[2],
    },
  });

  return `at://${h.data.did}/app.bsky.feed.post/${pathParts[4]}`;
};

In [ ]:
await postURLToAtpURI(
  "https://bsky.app/profile/callmephilip.com/post/3lcskn64bss2d",
);

"at://did:plc:ubdeopbbkbgedccgbum7dhsh/app.bsky.feed.post/3lcskn64bss2d"

# Grabbing thread data

Thread has a bunch of nested posts inside replies. Unwrap this into a list of
posts

In [ ]:
//| export

const unwrapThreadPosts = (
  thread: Thread,
): Post[] => {
  const posts: Post[] = [];

  // Add root post
  if (thread.post) {
    posts.push(thread.post);
  }

  // Recursively handle replies
  if (thread.replies) {
    thread.replies.forEach((reply) => {
      posts.push(...unwrapThreadPosts(reply as Thread));
    });
  }

  // Handle nested reply if present
  if ("parent" in thread && thread.parent) {
    posts.push(
      ...unwrapThreadPosts(thread.parent as Thread),
    );
  }

  return posts;
};

export const downloadThread = async (
  postUrl: string,
) => {
  const d = await rpc.get("app.bsky.feed.getPostThread", {
    params: {
      uri: await postURLToAtpURI(postUrl),
    },
  });
  return unwrapThreadPosts(d.data.thread as Thread);
};

In [ ]:
const posts = await downloadThread(
  "https://bsky.app/profile/callmephilip.com/post/3lcskn64bss2d",
);

# Converting posts to MD

The most basic setup is when there is just a piece of text display. More complex
cases will have links, attachments

In [ ]:
//| export

export const postToMd = (post: Post): string => {
  const record = post.record as PostRecord;
  const text = record.text;
  return `
    # ${post.author.displayName} (@${post.author.handle}) - ${record.createdAt}

    ${text}
  `;
};

In [ ]:
await Deno.jupyter.display(
  {
    "text/markdown": postToMd(posts[0]),
  },
  { raw: true },
);


    # Philip Nuzhnyi (@callmephilip.com) - 2024-12-08T15:46:20.768Z

    🧵 Notes on Deno + Jupyter 🦕 🔭
  

In [ ]:
await Deno.jupyter.display(
  {
    "text/markdown": posts.reduce((acc, post) => {
      return acc + postToMd(post);
    }, ""),
  },
  { raw: true },
);


    # Philip Nuzhnyi (@callmephilip.com) - 2024-12-08T15:46:20.768Z

    🧵 Notes on Deno + Jupyter 🦕 🔭
  
    # Philip Nuzhnyi (@callmephilip.com) - 2024-12-16T20:24:54.774Z

    useful reference - PR with Jupyter support for Deno github.com/denoland/den...
  
    # Philip Nuzhnyi (@callmephilip.com) - 2024-12-16T18:07:28.812Z

    As far as tests are concerned, ended up dumping them into a separate ts module and then using native `deno test`
  
    # Philip Nuzhnyi (@callmephilip.com) - 2024-12-16T18:06:10.698Z

    notebook cmd line issue is described here: github.com/jupyter/nbcl.... Includes a PR with a suggested fix. In the meantime, using some homemade stuff based on the nbclient fork - github.com/callmephilip...
  
    # Philip Nuzhnyi (@callmephilip.com) - 2024-12-16T18:03:11.209Z

    Hit 2 issues with Deno + Jupyter: running notebooks from cmd and tracking failing tests.
  
    # Philip Nuzhnyi (@callmephilip.com) - 2024-12-08T15:49:32.248Z

    Jupyter kernel for Deno - Deno docs docs.deno.com/runtime/refe...
  
    # Philip Nuzhnyi (@callmephilip.com) - 2024-12-08T15:56:32.679Z

    Repro of original Deno2 jupyter demo with broken mermaid integration - gist.github.com/aaronblondea...
  
    # Philip Nuzhnyi (@callmephilip.com) - 2024-12-08T16:00:14.858Z

    Rich displays for jupyter js kernels - deno.land/x/display@v1...
  
    # Philip Nuzhnyi (@callmephilip.com) - 2024-12-08T16:26:17.631Z

    @kylekelley.bsky.social has a bunch of Deno in notebook examples here - github.com/rgbkrk/denot...
  
    # Philip Nuzhnyi (@callmephilip.com) - 2024-12-08T16:44:33.927Z

    nice video showing deno + jupyter (via noteable RIP) www.youtube.com/watch?v=b5OK...
  
    # Philip Nuzhnyi (@callmephilip.com) - 2024-12-08T16:53:55.726Z

    anywidget for deno jupyter kernel - jsr.io/@anywidget/d...
  

In [ ]:
//| export

export const downloadPostToMd = async (postUrl: string): Promise<string> => {
  const posts = await downloadThread(postUrl);

  return posts.reduce((acc, post) => {
    return acc + postToMd(post);
  }, "");
};

In [ ]:
await downloadPostToMd(
  "https://bsky.app/profile/callmephilip.com/post/3lcskn64bss2d",
);

"\n" +
  "    # Philip Nuzhnyi (@callmephilip.com) - 2024-12-08T15:46:20.768Z\n" +
  "\n" +
  "    🧵 Notes on Deno + Jupyter 🦕 🔭\n" +
  "  \n" +
  "    # Philip Nuzhnyi (@callmephilip.com) - 2024-12-16T20:24:54.774Z\n" +
  "\n" +
  "    useful reference - PR with Jupyter support for Deno github.com/denoland/den...\n" +
  "  \n" +
  "    # Philip Nuzhnyi (@callmephilip.com) - 2024-12-16T18:07:28.812Z\n" +
  "\n" +
  "    As far as tests are concerned, ended up dumping them into a separate ts module and then using native `deno test`\n" +
  "  \n" +
  "    # Philip Nuzhnyi (@callmephilip.com) - 2024-12-16T18:06:10.698Z\n" +
  "\n" +
  "    notebook cmd line issue is described here: github.com/jupyter/nbcl.... Includes a PR with a suggested fix. In the meantime, using some homemade stuff based on the nbclient fork - github.com/callmephilip...\n" +
  "  \n" +
  "    # Philip Nuzhnyi (@callmephilip.com) - 2024-12-16T18:03:11.209Z\n" +
  "\n" +
  "    Hit 2 issues with Deno + Jupyter: running